<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Process-Data" data-toc-modified-id="Process-Data-1">Process Data</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-2">Dataset</a></span></li><li><span><a href="#Trainer" data-toc-modified-id="Trainer-3">Trainer</a></span></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-4">Train Model</a></span></li><li><span><a href="#One-layer" data-toc-modified-id="One-layer-5">One-layer</a></span></li><li><span><a href="#Two-layers" data-toc-modified-id="Two-layers-6">Two-layers</a></span></li></ul></div>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

In [2]:
# Set seed for reproducibility
seed = 9

-----------

# Process Data

In [3]:
# Load data
X, y = load_breast_cancer(return_X_y=True)
X.shape, y.shape

((569, 30), (569,))

In [4]:
# Convert np -> tensor
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)

In [5]:
# Train-test-split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train.shape, X_val.shape

(torch.Size([455, 30]), torch.Size([114, 30]))

In [6]:
# Normalize
mu, sigma = X_train.mean(), X_train.std()
X_train = (X_train - mu) / sigma
X_val = (X_val - mu) / sigma

----------

# Dataset

In [7]:
class TabularData(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y.unsqueeze(1)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.y)

--------

# Trainer

In [8]:
def accuracy(y, y_hat):
    """Compute accuracy given soft binary predictions."""
    y_pred = y_hat > 0.5
    n_correct = (y_pred == y).sum().item()
    return n_correct / len(y)

In [9]:
class PyTrainer:
    """Container for training a PyTorch feedforward neural net."""
    
    def __init__(self, model, optimizer, criterion, metric, train_dl, val_dl):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.metric = metric
        self.train_dl = train_dl
        self.val_dl = val_dl
        
    def _train(self):
        """Train for a single epoch and return the loss."""
        loss, n = 0, 0
        for x, y in self.train_dl:
            y_hat = self.model.forward(x)
            batch_loss = self.criterion(y_hat, y)
            self.optimizer.zero_grad()
            batch_loss.backward()
            self.optimizer.step()
            loss += len(y) * batch_loss.item()
            n += len(y)
        return loss / n
            
    def train(self, n_epochs, log_level=1):
        """Train for multiple epochs."""
        for epoch in range(n_epochs):
            self.model.train()
            loss = self._train()
            self.optimizer.step()
            val_loss, val_metric = self.evaluate(self.val_dl)
            if (epoch + 1) % log_level == 0:
                print(f"{epoch= :2d} | {loss= :.3f} | {val_loss= :.3f} | {val_metric= :.3f}")
    
    def evaluate(self, dl):
        """Return loss and metric on validation or test set."""
        self.model.eval()
        loss, n, metric = 0, 0, 0
        for x, y in dl:
            y_hat = self.model.forward(x)
            batch_loss = self.criterion(y_hat, y)
            batch_metric = self.metric(y, y_hat)
            metric += len(y) * batch_metric
            loss += len(y) * batch_loss.item()
            n += len(y)
        return loss / n, metric / n

----------

# Train Model

In [10]:
# Load training and validation data
train_ds = TabularData(X_train, y_train)
val_ds = TabularData(X_val, y_val)

batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=len(X_val), shuffle=False)

# One-layer 

In [11]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise model
metric = accuracy
criterion = nn.BCELoss()
model = nn.Sequential(nn.Linear(n_inp, 1), nn.Sigmoid())

# Initialise optimizer and trainer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
pytrainer = PyTrainer(model, optimizer, criterion, metric, train_dl, val_dl)

In [12]:
pytrainer.train(10)

epoch=  0 | loss= 0.609 | val_loss= 0.576 | val_metric= 0.596
epoch=  1 | loss= 0.548 | val_loss= 0.527 | val_metric= 0.798
epoch=  2 | loss= 0.504 | val_loss= 0.454 | val_metric= 0.895
epoch=  3 | loss= 0.458 | val_loss= 0.448 | val_metric= 0.825
epoch=  4 | loss= 0.430 | val_loss= 0.392 | val_metric= 0.895
epoch=  5 | loss= 0.403 | val_loss= 0.384 | val_metric= 0.895
epoch=  6 | loss= 0.387 | val_loss= 0.388 | val_metric= 0.851
epoch=  7 | loss= 0.379 | val_loss= 0.341 | val_metric= 0.886
epoch=  8 | loss= 0.353 | val_loss= 0.327 | val_metric= 0.904
epoch=  9 | loss= 0.344 | val_loss= 0.387 | val_metric= 0.877


# Two-layers

In [13]:
# Initialise model
metric = accuracy
criterion = nn.BCELoss()
model = nn.Sequential(nn.Linear(n_inp, 30), nn.ReLU(), nn.Linear(30, 1), nn.Sigmoid())

# Initialise optimizer and trainer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
pytrainer = PyTrainer(model, optimizer, criterion, metric, train_dl, val_dl)

In [14]:
pytrainer.train(20)

epoch=  0 | loss= 0.677 | val_loss= 0.634 | val_metric= 0.798
epoch=  1 | loss= 0.618 | val_loss= 0.578 | val_metric= 0.807
epoch=  2 | loss= 0.562 | val_loss= 0.516 | val_metric= 0.877
epoch=  3 | loss= 0.508 | val_loss= 0.467 | val_metric= 0.886
epoch=  4 | loss= 0.455 | val_loss= 0.509 | val_metric= 0.684
epoch=  5 | loss= 0.427 | val_loss= 0.371 | val_metric= 0.868
epoch=  6 | loss= 0.369 | val_loss= 0.418 | val_metric= 0.816
epoch=  7 | loss= 0.352 | val_loss= 0.296 | val_metric= 0.904
epoch=  8 | loss= 0.307 | val_loss= 0.503 | val_metric= 0.754
epoch=  9 | loss= 0.340 | val_loss= 0.346 | val_metric= 0.877
epoch= 10 | loss= 0.296 | val_loss= 0.295 | val_metric= 0.868
epoch= 11 | loss= 0.280 | val_loss= 0.291 | val_metric= 0.886
epoch= 12 | loss= 0.268 | val_loss= 0.234 | val_metric= 0.904
epoch= 13 | loss= 0.247 | val_loss= 0.323 | val_metric= 0.877
epoch= 14 | loss= 0.258 | val_loss= 0.229 | val_metric= 0.895
epoch= 15 | loss= 0.237 | val_loss= 0.217 | val_metric= 0.904
epoch= 1